In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#upload the data

In [ ]:
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path="/kaggle/working/")

In [ ]:

  
# loading the temp.zip and creating a zip object
with ZipFile("/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", 'r') as zObject_1:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject_1.extractall(
        path="/kaggle/working/")

In [ ]:
with ZipFile("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", 'r') as zObject_2:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject_2.extractall(
        path="/kaggle/working/")

In [ ]:
df_ts = pd.read_table("/kaggle/working/testData.tsv",on_bad_lines='skip')

In [ ]:
df_un = pd.read_table("/kaggle/working/unlabeledTrainData.tsv",on_bad_lines='skip')

In [ ]:
df = pd.read_table("/kaggle/working/labeledTrainData.tsv")

In [ ]:
#clean the reviews

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
train_data =[]
for idd in range(0,len(df.review)):
    example1 = BeautifulSoup(df["review"][idd]) 
    train_data.append(
        example1.get_text()
    )

In [ ]:
train_data_un =[]
for idd in range(0,len(df_un.review)):
    example1 = BeautifulSoup(df_un["review"][idd]) 
    train_data_un.append(
        example1.get_text()
    )

In [ ]:
test_data =[]
for idd in range(0,len(df_ts.review)):
    example1 = BeautifulSoup(df_ts["review"][idd]) 
    test_data.append(
        example1.get_text()
    )

In [ ]:
#попробовать позже как идея повышения качества
#train_data_new = []
#for idd in range(0,len(df.review)):
#    train_data_new.append(review_to_words(train_data[idd]))

In [ ]:
#cutoff - F1 maximization 

In [ ]:
#maximization F1
from scipy import optimize


def _accuracy(prob, true_labs, pred_prob, verbose=False):
    tp, tn, fp, fn = get_confusion_matrix(true_labs, pred_prob, prob[0])
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    if verbose:
        print("Probability: {0:0.4f} Accuracy: {1:0.4f}".format(prob[0], accuracy))
    return 1 - accuracy


def _f1(prob, true_labs, pred_prob, verbose=False):
    tp, tn, fp, fn = get_confusion_matrix(true_labs, pred_prob, prob[0])
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    if verbose:
        print("Probability: {0:0.4f} F1 score: {1:0.4f}".format(prob[0], f1))
    return 1 - f1


def get_confusion_matrix(true_labs, pred_prob, prob):
    pred_labs = pred_prob > prob
    # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
    tp = np.sum(np.logical_and(pred_labs == 1, true_labs == 1))
    # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
    tn = np.sum(np.logical_and(pred_labs == 0, true_labs == 0))    
    # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
    fp = np.sum(np.logical_and(pred_labs == 1, true_labs == 0))    
    # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
    fn = np.sum(np.logical_and(pred_labs == 0, true_labs == 1))

    return tp, tn, fp, fn


def get_probability(true_labs, pred_prob, objective='accuracy', verbose=False):
    if objective == 'accuracy':
        prob = optimize.brute(_accuracy, (slice(0.1, 0.9, 0.1),), args=(true_labs, pred_prob, verbose), disp=verbose)
    elif objective == 'f1':
        prob = optimize.brute(_f1, (slice(0.1, 0.9, 0.1),), args=(true_labs, pred_prob, verbose), disp=verbose)
    else:
        raise ValueError('`objective` must be `accuracy` or `f1`')
    return prob[0]

In [ ]:
#from sklearn.metrics import f1_score
#predict = (predictions >= 0.4875).astype(bool)
#f1_score(y_test, predict)

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
train_df = pd.DataFrame({"reviews": train_data})

In [ ]:
train_df_un = pd.DataFrame({"reviews": train_data_un})

In [ ]:
test_df = pd.DataFrame({"reviews": test_data})

In [ ]:
review_new=train_df.reviews.apply(gensim.utils.simple_preprocess)
review_new

In [ ]:
review_new_un=train_df_un.reviews.apply(gensim.utils.simple_preprocess)
review_new_un

In [ ]:
review_test=test_df.reviews.apply(gensim.utils.simple_preprocess)
review_test

In [ ]:
review_big =[]
for idd in range(0,len(df.review)):
    review_big.append(
        review_new[idd]
    )
for idd in range(0,len(df_un.review)):
    review_big.append(
        review_new_un[idd]
    )    
for idd in range(0,len(df_ts.review)):
    review_big.append(
        review_test[idd]
    )

In [ ]:
model2 = gensim.models.Word2Vec(review_big, min_count = 1,vector_size = 300, window = 6, sg=0)

In [ ]:
embading = []
for i in range(0,len(review_new)):
    a = sum(model2.wv[review_new[i]])/len(model2.wv[review_new[i]])
    embading.append(a)

In [ ]:
var = pd.DataFrame(embading)
var

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(var, df["sentiment"], 
                                                    test_size=0.2)

In [ ]:
import xgboost 
from xgboost import XGBClassifier

In [ ]:
xgb  = XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
predictions = xgb.predict_proba(X_test)[:,1]
roc_auc_score(y_test, predictions)

In [ ]:
p = get_probability(y_test, predictions, 'accuracy')
p

In [ ]:
xgb.fit( var, df["sentiment"] )

In [ ]:
embading_test = []
for i in range(0,len(review_test)):
    a = sum(model2.wv[review_test[i]])/len(model2.wv[review_test[i]])
    embading_test.append(a)

In [ ]:
var_test = pd.DataFrame(embading_test)

In [ ]:
predictions_t = RF.predict_proba(var_test)[:,1]

In [ ]:
t1 = pd.DataFrame({"pred": predictions_t, "sentiment": 0})

In [ ]:
for i in range(0,len(t)):
    if t1.pred[i] > 0.57:
        t1['sentiment'][i] = 1


In [ ]:
result = pd.merge(df_ts, t1, left_index=True, right_index=True)

In [ ]:
output = result.drop(['review','pred'], axis=1)

In [ ]:
output.to_csv( "Model_Submission_final.csv", index=False, quoting=3 )